# Reject Inference

In [ ]:
#@title **Location** of the dataset
path =  "drive/Shareddrives/H.IAAC - AI in Finance/Datasets/Raw Data/HomeCredit/"
process_path = "drive/MyDrive/Credit Project Prototypes/Datasets/Credit Bureau/ProcessedData/"

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import secrets
import optuna
import os
import math
from optuna.samplers import TPESampler
from scipy import stats
from pathlib import Path
from joblib import dump, load
import numpy as np
from scipy.stats import beta, ks_2samp

from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, LabelEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import IsolationForest
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import (ConfusionMatrixDisplay, accuracy_score, balanced_accuracy_score,
                             confusion_matrix, precision_recall_curve, auc,
                             roc_auc_score, roc_curve, recall_score,
                             classification_report, precision_score, recall_score, f1_score)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.semi_supervised import LabelSpreading

In [ ]:
#@title Read dataset
df_o = pd.read_csv(path+'application_train.csv')    #HomeCredit training dataset

df = pd.read_csv(process_path+'accepts_by_P4.csv')          #Accepted dataset by P4
df_r = pd.read_csv(process_path+'rejects_by_P4.csv')        #Rejected dataset by P4

# Create datasets with original columns but for selected samples for each dataset
df = df_o.loc[df_o.SK_ID_CURR.isin(df.SK_ID_CURR)]                    #Accepted dataset by P4
df_r = df_o.loc[df_o.SK_ID_CURR.isin(df_r.SK_ID_CURR)]                #Rejected dataset by P4

#Columns to use on RI study
cols_sel = ['SK_ID_CURR','TARGET', 'AMT_CREDIT', 'EXT_SOURCE_1', 'EXT_SOURCE_2',
        'EXT_SOURCE_3', 'REGION_POPULATION_RELATIVE', 'DAYS_EMPLOYED', 'DAYS_BIRTH', 'AMT_INCOME_TOTAL',
        'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'REG_CITY_NOT_WORK_CITY', 'AMT_GOODS_PRICE',
        'FLAG_OWN_CAR', 'NAME_EDUCATION_TYPE', 'NAME_CONTRACT_TYPE']

#Create versions of both datasets with only selected columns and samples selected by P4
df = df[cols_sel]
df_r = df_r[cols_sel]

In [ ]:
df, df_test = train_test_split(df, test_size=0.2, random_state=880,)

In [ ]:
# Open the file in read mode
with open('drive/Shareddrives/H.IAAC - AI in Finance/Datasets/Raw Data/Sentences/sentences.txt', 'r') as file:
    # Read the lines from the file into a list
    sentences_list = file.readlines()

# Remove trailing newline characters from each sentence
sentences_list = np.array([sentence.strip() for sentence in sentences_list])

In [ ]:
#@title Set seed
new_seed = False #@param {type:"boolean"}

if new_seed:
    seed_number = secrets.randbelow(1_000) #to name the results files

    while seed_number <100:
        seed_number = secrets.randbelow(1_000)
else:
    seed_number = 885

print(seed_number)

885


#Params

In [ ]:
params_dict = {
    'RandomForest_1' : {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None,
                        'criterion': 'entropy', 'max_depth': 10, 'max_features': 'log2',
                        'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0,
                        'min_samples_leaf': 9, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.0,
                        'n_estimators': 173, 'n_jobs': None, 'oob_score': False, 'random_state': seed_number,
                        'verbose': 0, 'warm_start': False},

    'RandomForest_2' : {'n_estimators': 113, 'criterion': 'entropy', 'max_depth': 9,
                        'min_samples_split': 9,'min_samples_leaf': 9,
                        'max_features': 'sqrt', 'bootstrap': False, 'random_state': seed_number,},

    'RandomForest_3' : {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None,
                        'criterion': 'entropy', 'max_depth': 10, 'max_features': 'auto',
                        'max_leaf_nodes': None, 'max_samples': None,
                        'min_impurity_decrease': 0.0, 'min_samples_leaf': 6,
                        'min_samples_split': 12, 'min_weight_fraction_leaf': 0.0,
                        'n_estimators': 168, 'n_jobs': None, 'oob_score': False,
                        'random_state': seed_number, 'verbose': 0, 'warm_start': False},

    'DecisionTree_1' : {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini',
                'max_depth': None, 'max_features': None, 'max_leaf_nodes': 25,
                'min_impurity_decrease': 0.0, 'min_samples_leaf': 100,
                'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0,
                'random_state': seed_number, 'splitter': 'best'},

    'LightGBM_1' : {'boosting_type': 'gbdt', 'class_weight': None,
                'colsample_bytree': 0.29280447167767465, 'importance_type': 'split',
                'learning_rate': 0.025693783679932296, 'max_depth': 6,
                'min_child_samples': 48, 'min_child_weight': 0.001,
                'min_split_gain': 0.0, 'n_estimators': 194, 'n_jobs': -1,
                'num_leaves': 35, 'objective': None, 'random_state': seed_number,
                'reg_alpha': 0.42692653558951865, 'reg_lambda': 0.7009056503658567,
                'verbose': -1, 'subsample': 0.9673695418639782,
                'subsample_for_bin': 200000, 'subsample_freq': 0,
                'is_unbalance': True},


    'LightGBM_2' : {'boosting_type': 'gbdt', 'class_weight': None,
              'colsample_bytree': 0.22534977954592625, 'importance_type': 'split',
              'learning_rate': 0.052227873762946964, 'max_depth': 5,
              'min_child_samples': 26, 'min_child_weight': 0.001,
              'min_split_gain': 0.0, 'n_estimators': 159, 'n_jobs': -1,
              'num_leaves': 12, 'objective': None, 'random_state': seed_number,
              'reg_alpha': 0.7438345471808012, 'reg_lambda': 0.46164693905368515,
                'verbose': -1, 'subsample': 0.8896599304061413,
              'subsample_for_bin': 200000, 'subsample_freq': 0,
              'is_unbalance': True},



    'LG_balanced' : { 'solver': 'liblinear','penalty': 'l1',
            'random_state': seed_number,'max_iter': 10000,   'class_weight': 'balanced'},

    'LG_1' : { 'solver': 'liblinear','penalty': 'l1',
            'random_state': seed_number,'max_iter': 10000, },

    'LabelSpreading_1' : {'alpha': 0.9212289329319412, 'gamma': 0.024244533484333246,
                        'kernel': 'knn', 'max_iter': 50, 'n_jobs': None,
                          'n_neighbors': 10, 'tol': 0.001, },
    'LabelSpreading_2' : {'alpha': 0.2, 'gamma': 20,
                        'kernel': 'knn', 'max_iter': 30, 'n_jobs': None,
                          'n_neighbors': 7, 'tol': 0.001,},

    }


#<font color='orange'>Helper Functions</font>


In [ ]:
#@title Function to return columns from the same type
def columns_by_type(dataframe, types_cols = ['numeric'], debug=False):
    list_cols = []
    if types_cols == ['numeric']:
        types_cols = ['b','i','u','f','c']
    elif types_cols == ['categorical']:
        types_cols = ['O', 'S', 'U']

    # Iterate through each column in the DataFrame
    for c in dataframe.columns:
        col = dataframe[c]

        # Check if the column's data type matches any of the specified data types
        if (col.dtype.kind in types_cols) or (col.dtype in types_cols):
            list_cols.append(c)

            # Print debugging information if debug flag is enabled
            if debug:
                print(c, ' : ', col.dtype)
                print(col.unique()[:10])
                print('---------------')

    return list_cols


In [ ]:
#@title Function to return the types of columns in the dataset
def types_columns(dataframe):
    listTypes = set()

    # Iterate through each column in the DataFrame
    for c in dataframe.columns:
        # Add the data type of the column to the set of unique data types
        listTypes.add(dataframe[c].dtype)

    # Convert the set of unique data types to a list
    return list(listTypes)


In [ ]:
#@title Compute and plot confusion matrix given true values and predictions
def plot_confusion_matrix(y_true, y_pred):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    print("Accuracy of the model is: ", accuracy_score(y_true, y_pred))

    # Plot confusion matrix
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [ ]:
# @title Compute and distributions of probs by target given true values and probabilities
def plot_prob_distribution(y_true, y_prob):
    plt.hist(y_prob[y_true==1], bins=20, alpha=0.5, label='1', density=True)
    plt.hist(y_prob[y_true==0], bins=20, alpha=0.5, label='0', density=True)
    plt.xlabel('Predicted probability')
    plt.ylabel('Frequency')
    plt.legend(loc='upper center')
    plt.show()

In [ ]:
#@title Compute and distributions of probs by target given true values and a list of probabilities
def plot_prob_distribution_from_list(y_true, y_prob_list):
    num_plots = len(y_prob_list)
    num_cols = math.ceil(math.sqrt(num_plots))
    num_rows = math.ceil(num_plots / num_cols)

    fig, axs = plt.subplots(num_rows, num_cols, figsize=(12, 8))

    for i, y_prob in enumerate(y_prob_list):
        row = i // num_cols
        col = i % num_cols

        if num_rows == 1 and num_cols == 1:
            axs.hist(y_prob[y_true == 1], bins=20, alpha=0.5, label='1', density=True)
            axs.hist(y_prob[y_true == 0], bins=20, alpha=0.5, label='0', density=True)
            axs.set_xlabel('Predicted probability')
            axs.set_ylabel('Frequency')
            axs.legend(loc='upper center')
        else:
            axs[row, col].hist(y_prob[y_true == 1], bins=20, alpha=0.5, label='1', density=True)
            axs[row, col].hist(y_prob[y_true == 0], bins=20, alpha=0.5, label='0', density=True)
            axs[row, col].set_xlabel('Predicted probability')
            axs[row, col].set_ylabel('Frequency')
            axs[row, col].legend(loc='upper center')

    # Hide unused subplots
    for i in range(len(y_prob_list), num_rows * num_cols):
        row = i // num_cols
        col = i % num_cols
        if num_rows == 1 and num_cols == 1:
            axs.axis('off')
        else:
            axs[row, col].axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
def calculate_kickout_metric(C1, C2, X_test, y_test, X_unl_holdout, acp_rate = 0.15):
    # Calculate predictions and obtain subsets A1_G and A1_B
    num_Acp_1 = int(len(X_test) * acp_rate) #number of Accepts
    y_prob_1 = C1.predict_proba(X_test)[:, 1]
    threshold = np.percentile(y_prob_1, 100 - (num_Acp_1 / len(y_prob_1)) * 100)
    y_pred_1 = (y_prob_1 > threshold).astype('int')
    A1 = X_test[y_pred_1 == 0]
    A1_G = X_test[(y_pred_1 == 0) & (y_test == 0)]
    A1_B = X_test[(y_pred_1 == 0) & (y_test == 1)]

    # Calculate predictions on X_test_holdout and obtain subset A2
    num_Acp_2 = int(len(X_test_holdout) * acp_rate) #number of Accepts
    y_prob_2 = C2.predict_proba(X_test_holdout)[:, 1]
    threshold = np.percentile(y_prob_2, 100 - (num_Acp_2 / len(y_prob_2)) * 100)
    y_pred_2 = (y_prob_2 > threshold).astype('int')
    A2 = X_test_holdout[y_pred_2 == 0]

    # Calculate indices of kicked-out good and bad samples
    indices_KG = np.setdiff1d(A1_G.index, A2.index)
    indices_KB = np.setdiff1d(A1_B.index, A2.index)

    # Calculate the count of kicked-out good and bad samples
    KG = A1_G.loc[indices_KG].shape[0]
    KB = A1_B.loc[indices_KB].shape[0]

    # Calculate the share of bad cases in A1
    p_B = A1_B.shape[0] / A1.shape[0]

    # Calculate the number of bad cases selected by the original model
    SB = A1_B.shape[0]

    # Calculate the kickout metric value
    kickout = ((KB / p_B) - (KG / (1 - p_B))) / (SB / p_B)

    return kickout, KG, KB

In [ ]:
def get_best_threshold_with_ks(model, X, y):
    y_probs = model.predict_proba(X)[:,1]
    fpr, tpr, thresholds = roc_curve(y, y_probs)
    return thresholds[np.argmax(tpr - fpr)]

In [ ]:
def risk_score_threshold(model, X, y, plot = False, defaul_acceped = 0.04):
    #calculate probabilities on validation set
    y_probs = model.predict_proba(X)[:,1]
    #sort index of probabilies on ascending order
    sorted_clients = np.argsort(y_probs)
    #calculate the comulative mean of the probabilities
    cum_mean = np.cumsum(y.iloc[sorted_clients])/np.arange(1, y.shape[0]+1)
    #turn to zero the first 1000 values to reduce noise
    cum_mean[:1000] = np.zeros(1000)
    #get the minimum threshold value that accepts until 4 percent default rate
    thr_0 = y_probs[sorted_clients][np.argmin(abs(cum_mean-defaul_acceped))]

    #plot the threshold x cum_mean graph
    if plot:
        plt.plot(y_probs[sorted_clients], cum_mean, '.')
    return thr_0

In [ ]:
def calculate_approval_rate(C1, X_val, y_val, X_test):
    threshold = risk_score_threshold(C1, X_val, y_val)
    y_prob = C1.predict_proba(X_test)[:,1]
    y_pred = (y_prob > threshold).astype(int)  # Convert probabilities to binary predictions
    n_approved = (y_pred == 0).sum()

    return n_approved/X_test.shape[0]


In [ ]:
#@title Function to get metrics for Reject Inference

def get_metrics_RI(name_model_dict, X, y, X_v = False, y_v = False,
                   X_unl_holdout = False, threshold_type = 'default', acp_rate = 0.15):
    models_dict = {}
    for name, model in name_model_dict.items():
        if type(model) == list:
            y_prob = model[0].predict_proba(X)[:,1]
            threshold_model = model[1]
            y_pred = (y_prob >= threshold_model).astype('int')
        else:
            if threshold_type == 'default':
                threshold = 0.5
            elif threshold_type == 'ks':
                if np.any(X_v):
                    threshold = get_best_threshold_with_ks(model, X_v, y_v)
                else:
                    threshold = get_best_threshold_with_ks(model, X, y)
            elif threshold_type == 'risk':
                if np.any(X_v):
                    threshold = risk_score_threshold(model, X_v, y_v)
                else:
                    threshold = risk_score_threshold(model, X, y)

            y_prob = model.predict_proba(X)[:,1]
            y_pred = (y_prob >= threshold).astype('int')

        models_dict[name] = (y_pred, y_prob)

    def evaluate_ks(y_real, y_proba):
        ks = ks_2samp(y_proba[y_real == 0], y_proba[y_real == 1])
        return ks.statistic

    def get_metrics_df(models_dict, y_true,):
        metrics_dict = {
            "Overall AUC": (
                lambda x: roc_auc_score(y_true, x), False),
            "KS": (
                lambda x: evaluate_ks(y_true, x), False),
            "------": (lambda x: "", True),
            "Balanced Accuracy": (
                lambda x: balanced_accuracy_score(y_true, x), True),
            "Accuracy": (
                lambda x: accuracy_score(y_true, x), True),
            "Precision": (
                lambda x: precision_score(y_true, x), True),
            "Recall": (
                lambda x: recall_score(y_true, x), True),
            "F1": (
                lambda x: f1_score(y_true, x), True),
            "-----": (lambda x: "", True),
        }
        df_dict = {}
        for metric_name, (metric_func, use_preds) in metrics_dict.items():
            df_dict[metric_name] = [metric_func(preds) if use_preds else metric_func(scores)
                                    for model_name, (preds, scores) in models_dict.items()]
        return df_dict

    df_dict = get_metrics_df(models_dict, y)
    if np.any(X_v) == False:
        if np.any(X_unl_holdout) == False or 'original' not in name_model_dict:
            del df_dict["-----"]

    if np.any(X_v):
        df_dict['Approval Rate'] = []
    if np.any(X_unl_holdout) and 'original' in name_model_dict:
        df_dict['Kickout'] = []
        df_dict['KG'] = []
        df_dict['KB'] = []

    for name, model in name_model_dict.items():
        if name != 'original':
            if type(model) == list:
                if np.any(X_v):
                    df_dict['Approval Rate'].append(calculate_approval_rate(model[0], X_v, y_v, X))
                if np.any(X_unl_holdout) and 'original' in name_model_dict:
                    kickout, kg, kb = calculate_kickout_metric(name_model_dict['original'][0], model[0], X, y, X_unl_holdout, acp_rate)
                    df_dict['Kickout'].append(kickout*10)
                    df_dict['KG'].append(kg)
                    df_dict['KB'].append(kb)
            else:
                if np.any(X_v):
                    df_dict['Approval Rate'].append(calculate_approval_rate(model, X_v, y_v, X))
                if np.any(X_unl_holdout) and 'original' in name_model_dict:
                    try:
                        original = name_model_dict['original'][0]
                        kickout, kg, kb = calculate_kickout_metric(original, model, X, y, X_unl_holdout, acp_rate)
                    except:
                        original = name_model_dict['original']
                        kickout, kg, kb = calculate_kickout_metric(original, model, X, y, X_unl_holdout, acp_rate)
                    df_dict['Kickout'].append(kickout*10)
                    df_dict['KG'].append(kg)
                    df_dict['KB'].append(kb)
        else:
            if np.any(X_v):
                if type(model) == list:
                    df_dict['Approval Rate'].append(calculate_approval_rate(model[0], X_v, y_v, X))
                else:
                    df_dict['Approval Rate'].append(calculate_approval_rate(model, X_v, y_v, X))
            if np.any(X_unl_holdout) and 'original' in name_model_dict:
                df_dict['Kickout'].append(0)
                df_dict['KG'].append(0)
                df_dict['KB'].append(0)

    metrics_df = pd.DataFrame.from_dict(df_dict, orient="index", columns=models_dict.keys())
    return metrics_df

# get_metrics_RI(models_dict_iter, X_test, y_test, X_acp_val, y_acp_val, X_unl_hold, threshold_type = 'ks', acp_rate=0.15)

In [ ]:
#@title function for get shapes
def get_shapes(arrays):
    shape_dict = {}
    import inspect

    variables_before = inspect.currentframe().f_back.f_locals
    def variable_to_str(var):
        for name, value in variables_before.items():
            if value is var:
                return name
    if isinstance(arrays, (list, tuple)):
        for arr in arrays:
            name = variable_to_str(arr)
            print(name,':', arr.shape)
            shape_dict[name] = arr.shape
    elif hasattr(arrays, 'shape'):
        name = variable_to_str(arrays)
        print(name,':', arrays.shape)
        shape_dict[name] = arrays.shape

    return shape_dict

In [ ]:
def get_index_cols(data, cols):
        index_cols = []
        for c in cols:
            i = data.columns.get_loc(c)
            index_cols.append(i)
        return index_cols

## Pipeline functions


In [ ]:
class EBE(BaseEstimator, TransformerMixin, ):
    def __init__(self, k=None, crit = 3):
        self.k = k
        self.crit = crit

    def fit(self, X, y):
        self.feature_names_in_ = []
        self.n_features, self.n_items  = X.shape[1], X.shape[0]
        self._aux_dict_main = {}
        self.unique_values = {col: X[col].unique() for col in X.columns}
        # print(self.unique_values)
        for i in range(self.n_features):
            Xi = X.iloc[:,i]
            # print(Xi.unique())
            X_name = X.iloc[:,i].name

            y = pd.Series(y, index = X.index)
            aux_dict = (pd.Series(y)
                        .groupby(Xi)
                        .agg(["mean", "count"])
                        .to_dict())
            self._aux_dict_main[X_name] = aux_dict, y.mean(), y.count()
            self.feature_names_in_.append(X_name)
        return self

    def transform(self, X, y=None):
        Xt_list = []
        for i in range(self.n_features):
            # if len(X.iloc[:,i].unique()) >= self.crit:
                Xi = X.iloc[:,i]
                X_name = X.iloc[:,i].name
                X_copy = Xi.copy()
                fit_unique = set(self.unique_values[X_name])
                X_unique = set(Xi.unique())
                # print(X_unique)
                unknown_values = list(X_unique - fit_unique)
                # print(unknown_values)
                X_copy = X_copy.replace(unknown_values, np.nan)
                print(X_copy.unique())
                group_ave = X_copy.replace(self._aux_dict_main[X_name][0]['mean'])
                group_count = X_copy.replace(self._aux_dict_main[X_name][0]['count'])

                Xt = ((group_ave * group_count + self.k * self._aux_dict_main[X_name][1]) / (self.k + group_count)).values.reshape(-1, 1)
                # print(np.array([Xi]).T)
            # else:
            #     print(X.values.shape)
                # Xt = 0
                Xt_list.append(Xt)
        Xt_array = np.hstack(Xt_list)
        # print(np.isnan(Xt_array).sum())
        return Xt_array

    def get_feature_names_out(self, input_features=None):
        if isinstance(input_features, pd.DataFrame):
            return [col for col in input_features.columns]
        else:
            return [col for col in input_features]

In [ ]:
class CustomNames(BaseEstimator, TransformerMixin):
    def fit(self, X, y):
        return self

    def transform(self, X):
        # Create a copy of X to avoid modifying the original DataFrame
        X_copy = X.copy()
        # print(X.columns)
        # Assign new column names to the DataFrame
        new_column_names = [col.split("__")[-1]  for col in X.columns]
        X_copy.columns = new_column_names

        return X_copy

    def get_feature_names_out(self, input_features=None):
        if isinstance(input_features, pd.DataFrame):
            return [col.split("__")[-1]  for col in input_features.columns]
        else:
            return [col.split("__")[-1]  for col in input_features]

In [ ]:
#@title function to create a Pipeline that uses **EBE** given **X,y** and a **classifier(model)**
def create_pipeline(X, y, classifier = None, onehot = True, onehotdrop = False,
                    normalize = True, do_EBE = False, crit = 0):
    num_cols = columns_by_type(X, ['numeric'])
    cat_cols = columns_by_type(X, ['categorical'])

    #1 Fill NaNs
    numeric_nan_fill_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='mean'))
    ])

    categorical_nan_fill_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent'))
    ])

    fill_pipe = ColumnTransformer(
        transformers=[
            ('num', numeric_nan_fill_transformer, num_cols),
            ('cat', categorical_nan_fill_transformer, cat_cols),
        ]
    )
    fill_pipe.set_output(transform="pandas")

    #2: Ordinal encoder

    encoder_pipe = ColumnTransformer(
        transformers=[
            ('num', "passthrough", make_column_selector(dtype_include=np.number)),
                ('cat', EBE(crit) if do_EBE else OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = np.nan),
              make_column_selector('cat')),
        ]
    )

    encoder_pipe.set_output(transform="pandas")

    #3 Onehot encoder

    onehot_pipe = ColumnTransformer(
        transformers=[
        ('onehot_encoder', OneHotEncoder(drop='if_binary',
                                         sparse_output=False, handle_unknown='ignore') if onehotdrop else
                            OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
         make_column_selector('cat')),
    ], remainder = 'passthrough')

    onehot_pipe.set_output(transform="pandas")

    #4 Scalling

    scaling_pipe = ColumnTransformer([
        ('scaler', StandardScaler(), make_column_selector('num')),
    ], remainder = 'passthrough')

    # scaling_pipe.set_output(transform="pandas")

    #5 Better names

    name_pipe = Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
    ])

    # Combine all the transformers in a single pipeline
    pipeline = Pipeline(
        steps=[
            ('fill', fill_pipe),
            ('le', encoder_pipe),
            ('hot', onehot_pipe if onehot else 'passthrough'),
            ('ss', scaling_pipe if normalize else 'passthrough'),
            ('name', name_pipe),
            ('classifier', classifier if classifier is not None else 'passthrough'),
        ],
    )

    return pipeline

## Definition of Train and Test Val, and Unl

In [ ]:
X_acp = df.iloc[:, (df.columns != "TARGET") & (df.columns != "SK_ID_CURR")]
y_acp = df["TARGET"]

In [ ]:
X_unl = df_r.iloc[:, (df_r.columns != "TARGET") & (df_r.columns != "SK_ID_CURR")]
y_unl = df_r["TARGET"]

In [ ]:
X_unl, X_unl_val, y_unl, y_unl_val =  train_test_split(X_unl, y_unl, test_size=0.05, random_state=seed_number, stratify=y_unl)   #UNL and UNL_VAL
X_acp, X_acp_val, y_acp, y_acp_val =  train_test_split(X_acp, y_acp, test_size=0.2, random_state=seed_number, stratify=y_acp)   #ACP_VAL

X_train, X_test, y_train, y_test = train_test_split(X_acp, y_acp, test_size = 0.2, random_state=seed_number, stratify=y_acp)    #TRAIN
X_unl, X_unl_hold, y_unl, y_unl_hold = train_test_split(X_unl, y_unl, test_size=0.2, random_state=seed_number, stratify=y_unl)

In [ ]:
X_val = pd.concat([X_acp_val, X_unl_val])
y_val = np.concatenate([y_acp_val, y_unl_val])

X_val, X_val_test, y_val, y_val_test = train_test_split(X_val, y_val, test_size = 0.5, random_state=seed_number, stratify=y_val)

In [ ]:
get_shapes([X_unl, X_unl_val, X_train])
get_shapes([X_test])
get_shapes([X_val, X_val_test])

X_unl : (124900, 15)
X_unl_val : (8218, 15)
X_train : (32392, 15)
X_test : (8099, 15)
X_val : (9170, 15)
X_val_test : (9171, 15)


{'X_val': (9170, 15), 'X_val_test': (9171, 15)}

In [ ]:
X_test_holdout = pd.concat([X_test, X_unl_hold])

## PIPELINE NORMAL


In [ ]:
models_dict = {}

In [ ]:
clf_original = create_pipeline(X_train, y_train, LGBMClassifier(**params_dict['LightGBM_2']),)
clf_original.fit(X_train, y_train)

models_dict['original'] = clf_original
get_metrics_RI(models_dict, X_test, y_test,)

,original
Overall AUC,0.741476
KS,0.359283
------,
Balanced Accuracy,0.67607
Accuracy,0.70589
Precision,0.121922
Recall,0.642259
F1,0.20494


#<font color='green'>Test RI strategies:</font>

###<font color='darkorange'>Augmentation with Soft Cutoff:</font>
(Siddiqi, 2012)

In [ ]:
#@title Create Dataset
#Create dataset based on Approved(1)/Decline(0) condition
X_aug_train = pd.concat([X_train, X_unl])

train_y = np.ones(X_train.shape[0]) #Approved gets 1
unl_y = np.zeros(X_unl.shape[0])    #Rejected get 0

#Concat train_y and unl_y
y_aug_train = pd.Series(np.concatenate([train_y, unl_y]), index = X_aug_train.index)

In [ ]:
#@title Get Weights
classifier_AR = create_pipeline(X_aug_train, y_aug_train, LGBMClassifier(**params_dict['LightGBM_2']))
classifier_AR.fit(X_aug_train, y_aug_train)

#Get the probabilitie of being approved
prob_A = classifier_AR.predict_proba(X_aug_train)[:,1]
prob_A_series = pd.Series(prob_A, index = X_aug_train.index)

n_scores_interv = 100

#Sort the probabilities of being accepted
asc_prob_A = np.argsort(prob_A)
asc_prob_A_series = prob_A_series.iloc[asc_prob_A]
# #Split the probs in intervals
score_interv = np.array_split(asc_prob_A_series,n_scores_interv)

#Create array for accepts weights
weights_SC = y_train.copy()
for s in score_interv:
    #Get index of accepts in s
    acceptees = np.intersect1d(s.index, weights_SC.index)
    if len(acceptees) >= 1:
        #Augmentation Factor (Weight) for the split
        AF = y_aug_train.loc[s.index].mean() #A/(A+R)
        AF_split = np.power(AF ,-1)
        weights_SC.loc[acceptees] = AF_split

In [ ]:
#@title Fit classifier Soft Cutoff
augmentation_classifier_SC = create_pipeline(X_train, y_train, LGBMClassifier(**params_dict['LightGBM_2']))
augmentation_classifier_SC.fit(X_train, y_train, classifier__sample_weight = weights_SC)

models_dict['aug_SC'] = augmentation_classifier_SC

In [ ]:
get_metrics_RI(models_dict, X_test, y_test, X_acp_val, y_acp_val, X_unl_hold)

,original,aug_SC
Overall AUC,0.741476,0.734707
KS,0.359283,0.354836
------,,
Balanced Accuracy,0.67607,0.672156
Accuracy,0.70589,0.709594
Precision,0.121922,0.121567
Recall,0.642259,0.629707
F1,0.20494,0.203791
-----,,
Approval Rate,0.853562,0.865292


###<font color='darkorange'>Augmentation (Upward and Downward):</font>
[Augmentation,Reweighting] (Anderson, 2022)


[Augmentation] (Siddiqi, 2012)

In [ ]:
#@title Create Dataset
#Create dataset based on Approved(1)/Decline(0) condition
X_aug_train = pd.concat([X_train, X_unl])

train_y = np.ones(X_train.shape[0]) #Approved gets 1
unl_y = np.zeros(X_unl.shape[0])    #Rejected get 0

#Concat train_y and unl_y
y_aug_train = pd.Series(np.concatenate([train_y, unl_y]), index = X_aug_train.index)

In [ ]:
#@title Get weights
weight_classifier = create_pipeline(X_aug_train, y_aug_train, LGBMClassifier(**params_dict['LightGBM_2']))
weight_classifier.fit(X_aug_train, y_aug_train)

#Weights are the probabilitie of being approved
weights = weight_classifier.predict_proba(X_aug_train)[:,1]

#Upward: ŵ = w/p(A)
acp_weights_up = 1/weights[:X_train.shape[0]]

#Downward: ŵ = w * (1 - p(A))
acp_weights_down = 1 * (1 - weights[:X_train.shape[0]])

In [ ]:
#@title Fit classifier Upward
augmentation_classifier_up = create_pipeline(X_train, y_train, LGBMClassifier(**params_dict['LightGBM_2']))
augmentation_classifier_up.fit(X_train, y_train, classifier__sample_weight = acp_weights_up)

models_dict['aug_up'] = augmentation_classifier_up

In [ ]:
#@title Fit classifier Downward
augmentation_classifier_down = create_pipeline(X_train, y_train, LGBMClassifier(**params_dict['LightGBM_2']))
augmentation_classifier_down.fit(X_train, y_train, classifier__sample_weight = acp_weights_down)

models_dict['aug_down'] = augmentation_classifier_down

In [ ]:
get_metrics_RI(models_dict, X_test, y_test, X_acp_val, y_acp_val, X_unl_hold)

,original,aug_SC,aug_up,aug_down
Overall AUC,0.741476,0.734707,0.742237,0.741728
KS,0.359283,0.354836,0.363791,0.357673
------,,,,
Balanced Accuracy,0.67607,0.672156,0.673923,0.672728
Accuracy,0.70589,0.709594,0.711075,0.701445
Precision,0.121922,0.121567,0.122466,0.119906
Recall,0.642259,0.629707,0.631799,0.640167
F1,0.20494,0.203791,0.205163,0.20198
-----,,,,
Approval Rate,0.853562,0.865292,0.864181,0.861588


###<font color='darkorange'>Fuzzy Augmentation:</font>
[Fuzzy-Parcelling](Anderson, 2022)

[Code Inspiration](https://www.mathworks.com/help/risk/reject-inference-for-credit-scorecards.html)


In [ ]:
#@title Create Dataset
X_fuzzy_train = pd.concat([X_train, X_unl, X_unl])
X_fuzzy_train.index = range(X_fuzzy_train.shape[0])
good_y = np.zeros(X_unl.shape[0])
bad_y = np.ones(X_unl.shape[0])

y_fuzzy_rej = pd.Series(np.concatenate([good_y, bad_y]))
y_fuzzy_train = pd.concat([y_train, y_fuzzy_rej])
y_fuzzy_train.index = range(X_fuzzy_train.shape[0])

get_shapes([X_fuzzy_train, y_fuzzy_train])

X_fuzzy_train : (282192, 15)
y_fuzzy_train : (282192,)


{'X_fuzzy_train': (282192, 15), 'y_fuzzy_train': (282192,)}

In [ ]:
#@title Get weights
weight_clf = create_pipeline(X_train, y_train, LGBMClassifier(**params_dict['LightGBM_2']))
weight_clf.fit(X_train, y_train)
unl_0_weights = weight_clf.predict_proba(X_unl)[:,0]
unl_1_weights = weight_clf.predict_proba(X_unl)[:,1]

train_weights = np.ones(y_train.shape[0])

fuzzy_weights = np.concatenate([train_weights, unl_0_weights, unl_1_weights])

In [ ]:
#@title Fit classifier
fuzzy_classifier = create_pipeline(X_fuzzy_train, y_fuzzy_train, LGBMClassifier(**params_dict['LightGBM_2']))
fuzzy_classifier.fit(X_fuzzy_train, y_fuzzy_train, classifier__sample_weight = fuzzy_weights)

models_dict['fuzzy'] = fuzzy_classifier

get_metrics_RI(models_dict, X_test, y_test, X_acp_val, y_acp_val, X_unl_hold)

,original,aug_SC,aug_up,aug_down,fuzzy
Overall AUC,0.741476,0.734707,0.742237,0.741728,0.724491
KS,0.359283,0.354836,0.363791,0.357673,0.34783
------,,,,,
Balanced Accuracy,0.67607,0.672156,0.673923,0.672728,0.661738
Accuracy,0.70589,0.709594,0.711075,0.701445,0.79516
Precision,0.121922,0.121567,0.122466,0.119906,0.146195
Recall,0.642259,0.629707,0.631799,0.640167,0.51046
F1,0.20494,0.203791,0.205163,0.20198,0.227294
-----,,,,,
Approval Rate,0.853562,0.865292,0.864181,0.861588,0.855291


###<font color='darkorange'>Extrapolation:</font>
[hard cutoff, Simple Augmentation] (Siddiqi, 2012)

In [ ]:
#@title Fit classifier on Accepts Performance
default_classifier = create_pipeline(X_train, y_train, LGBMClassifier(**params_dict['LightGBM_2']))
default_classifier.fit(X_train, y_train)

y_prob_unl = default_classifier.predict_proba(X_unl)[:,1]
y_prob_unl_s = pd.Series(y_prob_unl, index = X_unl.index)

y_label_unl = (y_prob_unl >= 0.5).astype(int)
y_label_unl_s = pd.Series(y_label_unl, index = X_unl.index)

In [ ]:
#@title Create new Dataset
augmentation_type = "only_1" # @param ["only_1", "all", "confident"]

if augmentation_type == "only_1":
    new_X_train = pd.concat([X_train,X_unl[y_label_unl == 1]])
    new_y_train = pd.concat([y_train,y_label_unl_s[y_label_unl == 1]])
elif augmentation_type == "all":
    new_X_train = pd.concat([X_train,X_unl])
    new_y_train = pd.concat([y_train,y_label_unl_s])
elif augmentation_type == "confident":
    new_X_train = pd.concat([X_train,X_unl[y_prob_unl>0.8], X_unl[y_prob_unl<0.15]])
    new_y_train = pd.concat([y_train,y_label_unl_s[y_prob_unl>0.8], y_label_unl_s[y_prob_unl<0.15]])

#@title Fit classifier
extrap_classifier = create_pipeline(new_X_train, new_y_train, LGBMClassifier(**params_dict['LightGBM_2']))
extrap_classifier.fit(new_X_train, new_y_train)

models_dict['extrap'+'-'+augmentation_type] = extrap_classifier

In [ ]:
#@title Create new Dataset
augmentation_type = "all" # @param ["only_1", "all", "confident"]

if augmentation_type == "only_1":
    new_X_train = pd.concat([X_train,X_unl[y_label_unl == 1]])
    new_y_train = pd.concat([y_train,y_label_unl_s[y_label_unl == 1]])
elif augmentation_type == "all":
    new_X_train = pd.concat([X_train,X_unl])
    new_y_train = pd.concat([y_train,y_label_unl_s])
elif augmentation_type == "confident":
    new_X_train = pd.concat([X_train,X_unl[y_prob_unl>0.8], X_unl[y_prob_unl<0.15]])
    new_y_train = pd.concat([y_train,y_label_unl_s[y_prob_unl>0.8], y_label_unl_s[y_prob_unl<0.15]])

#@title Fit classifier
extrap_classifier = create_pipeline(new_X_train, new_y_train, LGBMClassifier(**params_dict['LightGBM_2']))
extrap_classifier.fit(new_X_train, new_y_train)

models_dict['extrap'+'-'+augmentation_type] = extrap_classifier

In [ ]:
#@title Create new Dataset
augmentation_type = "confident" # @param ["only_1", "all", "confident"]

if augmentation_type == "only_1":
    new_X_train = pd.concat([X_train,X_unl[y_label_unl == 1]])
    new_y_train = pd.concat([y_train,y_label_unl_s[y_label_unl == 1]])
elif augmentation_type == "all":
    new_X_train = pd.concat([X_train,X_unl])
    new_y_train = pd.concat([y_train,y_label_unl_s])
elif augmentation_type == "confident":
    new_X_train = pd.concat([X_train,X_unl[y_prob_unl>0.8], X_unl[y_prob_unl<0.15]])
    new_y_train = pd.concat([y_train,y_label_unl_s[y_prob_unl>0.8], y_label_unl_s[y_prob_unl<0.15]])

#@title Fit classifier
extrap_classifier = create_pipeline(new_X_train, new_y_train, LGBMClassifier(**params_dict['LightGBM_2']))
extrap_classifier.fit(new_X_train, new_y_train)

models_dict['extrap'+'-'+augmentation_type] = extrap_classifier

In [ ]:
get_metrics_RI(models_dict, X_test, y_test, X_acp_val, y_acp_val, X_unl_hold)

,original,aug_SC,aug_up,aug_down,fuzzy,extrap-only_1,extrap-all,extrap-confident
Overall AUC,0.741476,0.734707,0.742237,0.741728,0.724491,0.719902,0.730969,0.740082
KS,0.359283,0.354836,0.363791,0.357673,0.34783,0.345871,0.349946,0.358178
------,,,,,,,,
Balanced Accuracy,0.67607,0.672156,0.673923,0.672728,0.661738,0.655932,0.669765,0.632656
Accuracy,0.70589,0.709594,0.711075,0.701445,0.79516,0.767626,0.723546,0.862205
Precision,0.121922,0.121567,0.122466,0.119906,0.146195,0.132461,0.1242,0.179074
Recall,0.642259,0.629707,0.631799,0.640167,0.51046,0.529289,0.608787,0.372385
F1,0.20494,0.203791,0.205163,0.20198,0.227294,0.211893,0.20631,0.241848
-----,,,,,,,,
Approval Rate,0.853562,0.865292,0.864181,0.861588,0.855291,0.824176,0.851587,0.863687


###<font color='darkorange'>Label Spreading:</font>
[Label Spreading] (Zhou, 2004)(Kang, 2021)

In [ ]:
#@title Create dataset with Approved and Rejected
X_train_ls, y_train_ls, X_unl_ls = X_train.copy(), y_train.copy(), X_unl.copy()

X_combined = pd.concat([X_train_ls, X_unl_ls])

y_unl_ls = np.array([-1]*X_unl_ls.shape[0])
y_combined = np.concatenate([y_train_ls.array, y_unl_ls])
y_combined = pd.Series(y_combined, index=X_combined.index)

n_labeled_points = y_train_ls.shape[0]
indices = np.arange(y_combined.shape[0])

In [ ]:
#@title Predict labels on the unlabeled data
lp_model = create_pipeline(X_combined, y_combined, LabelSpreading(**params_dict['LabelSpreading_2']))
lp_model.fit(X_combined, y_combined)
predicted_labels = lp_model['classifier'].transduction_[indices[n_labeled_points:]]

y_label_pred_s = pd.Series(predicted_labels, index=X_unl_ls.index)


In [ ]:
# Create a new classifier pipeline using labeled and unlabeled data, and fit it
new_X_train = pd.concat([X_train, X_unl_ls])
new_y_train = pd.concat([y_train, y_label_pred_s])

clf_LS = create_pipeline(new_X_train, new_y_train, LGBMClassifier(**params_dict['LightGBM_2']))
clf_LS.fit(new_X_train, new_y_train,)

models_dict['labelSpr'] = clf_LS

In [ ]:
metrics_ri = get_metrics_RI(models_dict, X_test, y_test, X_acp_val, y_acp_val, X_unl_hold)
metrics_ri

,original,aug_SC,aug_up,aug_down,fuzzy,extrap-only_1,extrap-all,extrap-confident,labelSpr
Overall AUC,0.741476,0.734707,0.742237,0.741728,0.724491,0.719902,0.730969,0.740082,0.728476
KS,0.359283,0.354836,0.363791,0.357673,0.34783,0.345871,0.349946,0.358178,0.336808
------,,,,,,,,,
Balanced Accuracy,0.67607,0.672156,0.673923,0.672728,0.661738,0.655932,0.669765,0.632656,0.653498
Accuracy,0.70589,0.709594,0.711075,0.701445,0.79516,0.767626,0.723546,0.862205,0.792567
Precision,0.121922,0.121567,0.122466,0.119906,0.146195,0.132461,0.1242,0.179074,0.141408
Recall,0.642259,0.629707,0.631799,0.640167,0.51046,0.529289,0.608787,0.372385,0.495816
F1,0.20494,0.203791,0.205163,0.20198,0.227294,0.211893,0.20631,0.241848,0.220056
-----,,,,,,,,,
Approval Rate,0.853562,0.865292,0.864181,0.861588,0.855291,0.824176,0.851587,0.863687,0.86023
